In [1]:
import importlib
from tqdm import tqdm
import json
import pandas as pd
calculate_doc_term_recall = importlib.import_module('train-deep-ct').calculate_doc_term_recall
calculate_qtokens = importlib.import_module('train-deep-ct').calculate_qtokens

!nosetests
from util import clean_qtokens

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [2]:
DIR = '/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-probst/deep-ct/'
anchorText = {}

try:
    anchorText = clean_qtokens(json.load(open(DIR + 'ms-marco-trainig-set-deepct-qtokens.json', 'r')))
except Exception as e:
    print(e)
    print('Comment the following lines to create ms-marco-trainig-set-deepct-qtokens.json.')
    #with open(DIR + 'msmarco-document-train.jsonl') as f:
    #    for l in tqdm(f):
    #        l = json.loads(l)
    #        for target_doc in l['target_document']:
    #            if target_doc not in anchorText:
    #                anchorText[target_doc] = []
    #            anchorText[target_doc] += [l['contents']]
    #
    #for target_doc in tqdm([i for i in anchorText.keys()]):
    #    anchorText[target_doc] = calculate_qtokens(anchorText[target_doc])
    #json.dump(anchorText, open(DIR + 'ms-marco-trainig-set-deepct-qtokens.json', 'w'))

From qtokens with 319927 documents, I filtered terms that may cause train/test overlap so that I now have 249046 documents.


In [3]:
len(anchorText)

249046

In [4]:
[i for i in anchorText.keys()][:10]

['D59219',
 'D59235',
 'D576811',
 'D576840',
 'D114789',
 'D576852',
 'D576861',
 'D576889',
 'D255953',
 'D576909']

In [5]:
anchorText['D59219']

{'project': 1.0,
 'manhattan': 1.0,
 'immedi': 1.0,
 'success': 1.0,
 'impact': 1.0}

In [6]:
with open(DIR + 'msmarco-document-passages.jsonl') as docs, open(DIR + 'deep-ct-training-data-ms-marco-training-set-test-overlap-removed.jsonl', 'w') as out_file:
    for doc in tqdm(docs):
        doc = json.loads(doc)
        doc_id = doc['id']
        if doc_id not in anchorText:
            continue
        for passage_id, passage in doc['passages']:
            for i in calculate_doc_term_recall([{"doc":{"title": passage}}], anchorText[doc_id]):
                i['doc']['position'] = passage_id
                i['doc']['id'] = doc_id + '___' + str(passage_id)
                out_file.write(json.dumps(i) + '\n')

3213835it [1:03:08, 848.25it/s] 


In [15]:
saved_docs = []
examples = 0

with open(DIR + 'deep-ct-training-data-ms-marco-training-set-test-overlap-removed.jsonl', 'r') as docs:
    for l in docs:
        l = json.loads(l)
        examples += 1
        saved_docs += [l['doc']['id'].split('___')[0]]

print('Examples: ', examples)
print('Covered Documents: ', len(set(saved_docs)))

Examples:  2079858
Covered Documents:  249046


# Correlations

In [54]:
def correlation(a, b, correlation):
    import pandas as pd
    import json
    # TODO: Add all tokens with weight of 0?
    tokens = set([i for i in a.keys()] + [i for i in b.keys()])
    
    df_corr = []
    for token in tokens:
        df_corr += [{'word': token, 'weight_a': a.get(token, 0), 'weight_b': b.get(token, 0)}]
    
    ret = pd.DataFrame(df_corr).corr(correlation)
    if len(ret) != 2:
        raise ValueError('Could not handle \n' + json.dumps(a) + '\n' + json.dumps(b))
    ret = ret.iloc[0]

    if ret.name != 'weight_a':
        raise ValueError('Could not handle \n' + json.dumps(a) + '\n' + json.dumps(b))
    
    return float(ret['weight_b'])

def jaccard_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(set(list1)) + len(set(list2))) - intersection
    return float(intersection) / union

In [30]:
correlation({'project': 0.3, 'manhatten': 0.7, 'success': 1.0}, {'project': 1.0, 'manhatten': 0.5, 'success': 0.1})

-1.0

In [31]:
correlation({'project': 0.3, 'manhatten': 0.7, 'success': 1.0}, {'project': .5, 'manhatten': 2, 'success': 0.1})

-0.33333333333333337

In [55]:
jaccard_similarity(['a', 'b', 'c'], ['a', 'b', 'c'])

1.0

In [56]:
jaccard_similarity(['a', 'b', 'c'], ['a', 'b'])

0.6666666666666666

In [57]:
def calculate_correlation(method_a, method_b):
    import json
    from tqdm import tqdm
    doc_terms_method_a = json.load(open(DIR + method_a))
    doc_terms_method_b = json.load(open(DIR + method_b))
    ret = []
    
    for doc in tqdm([i for i in doc_terms_method_a.keys()]):
        if  doc not in doc_terms_method_b or len(doc_terms_method_a[doc].keys()) == 0 or len(doc_terms_method_b[doc].keys()) == 0:
            continue
            
        ret += [
            {
                'doc': doc,
                'kendall': correlation(doc_terms_method_a[doc], doc_terms_method_b[doc]  ,'kendall'),
                'pearson': correlation(doc_terms_method_a[doc], doc_terms_method_b[doc]  ,'pearson'),
                'jaccard': jaccard_similarity(doc_terms_method_a[doc].keys(), doc_terms_method_b[doc].keys())
            }
        ]
    
    return ret

In [62]:
df_corr_orcas_anchor = pd.DataFrame(calculate_correlation('ms-marco-orcas-deepct-qtokens.json', 'cc-2019-47-deepct-qtokens.json'))
df_corr_orcas_anchor

100%|██████████| 1147461/1147461 [22:52<00:00, 836.28it/s] 


,doc,kendall,pearson,jaccard
0,D84115,0.774597,0.996347,0.250000
1,D920978,-0.141266,0.264215,0.160000
2,D2577639,-0.840168,-0.441415,0.000000
3,D579719,-0.124788,0.990972,0.135338
4,D1094795,-0.284557,0.538009,0.144737
...,...,...,...,...
648321,D3011725,-1.000000,-1.000000,0.000000
648322,D1687148,NaN,NaN,0.333333
648323,D1093699,-0.617213,-0.246718,0.000000
648324,D847946,-0.845154,-0.737154,0.000000


In [64]:
df_corr_orcas_anchor.describe()

,kendall,pearson,jaccard
count,615983.000000,616198.000000,648326.000000
mean,-0.100523,0.244518,0.218530
std,0.472014,0.598377,0.186786
min,-1.000000,-1.000000,0.000000
25%,-0.412313,-0.213213,0.100000
50%,-0.109422,0.405584,0.176471
75%,0.200000,0.763133,0.285714
max,1.000000,1.000000,1.000000


In [61]:
df_corr_orcas_marco_training = pd.DataFrame(calculate_correlation('ms-marco-trainig-set-deepct-qtokens.json', 'ms-marco-orcas-deepct-qtokens.json'))
df_corr_orcas_marco_training

100%|██████████| 319927/319927 [04:15<00:00, 1252.45it/s]


,doc,kendall,pearson,jaccard
0,D59235,-0.401559,0.326956,0.181818
1,D576840,-1.000000,-1.000000,0.000000
2,D389790,-0.202189,-0.025318,0.062500
3,D576852,-0.106600,-0.171499,0.166667
4,D576861,NaN,NaN,1.000000
...,...,...,...,...
130714,D576666,-0.547723,-0.457693,0.000000
130715,D576680,0.267261,0.181071,0.333333
130716,D576733,NaN,NaN,0.666667
130717,D135542,0.072696,0.361305,0.052632


In [65]:
df_corr_orcas_marco_training.describe()

,kendall,pearson,jaccard
count,113293.000000,113303.000000,130719.000000
mean,-0.047983,0.085299,0.321577
std,0.496645,0.563068,0.243169
min,-1.000000,-1.000000,0.000000
25%,-0.427352,-0.408248,0.142857
50%,0.000000,0.141421,0.250000
75%,0.290659,0.563884,0.500000
max,1.000000,1.000000,1.000000


In [58]:
df_corr_anchor_marco_training = pd.DataFrame(calculate_correlation('ms-marco-trainig-set-deepct-qtokens.json', 'cc-2019-47-deepct-qtokens.json'))
df_corr_anchor_marco_training

100%|██████████| 319927/319927 [04:39<00:00, 1144.28it/s]


,doc,kendall,pearson,jaccard
0,D59219,-1.000000,-1.000000,0.000000
1,D59235,-0.362459,0.393819,0.076923
2,D576840,-1.000000,-1.000000,0.000000
3,D576852,-0.223495,-0.197392,0.066667
4,D275258,-0.577350,-0.577350,0.250000
...,...,...,...,...
132701,D576620,-0.456859,-0.448696,0.200000
132702,D576733,NaN,NaN,0.666667
132703,D135542,-0.466371,-0.294740,0.115385
132704,D576747,NaN,NaN,1.000000


In [66]:
df_corr_anchor_marco_training.describe()

,kendall,pearson,jaccard
count,117218.000000,117262.000000,132706.000000
mean,-0.258035,-0.083936,0.197573
std,0.501333,0.625695,0.212131
min,-1.000000,-1.000000,0.000000
25%,-0.659966,-0.632456,0.050000
50%,-0.258199,-0.082578,0.133333
75%,0.059235,0.467464,0.266667
max,1.000000,1.000000,1.000000
